<a href="https://colab.research.google.com/github/Orthogonal-Research-Lab/Neuromatch-Academy/blob/master/IBL_dataloading.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import os
import re
from IPython.display import clear_output
from matplotlib import pyplot as plt
import numpy as np
import pandas as pd

# Install then import PsyTrack
!pip install psytrack==1.3
import psytrack as psy

# Set save path for all figures, decide whether to save permanently
SPATH = "ColabFigureData/"
!mkdir -p "{SPATH}"

# Set matplotlib defaults for making files consistent in Illustrator
colors = psy.COLORS
zorder = psy.ZORDER
plt.rcParams['figure.dpi'] = 140
plt.rcParams['savefig.dpi'] = 300
plt.rcParams['savefig.facecolor'] = (1,1,1,0)
plt.rcParams['savefig.bbox'] = "tight"
plt.rcParams['font.size'] = 10
# plt.rcParams['font.family'] = 'sans-serif'     # not available in Colab
# plt.rcParams['font.sans-serif'] = 'Helvetica'  # not available in Colab
plt.rcParams['pdf.fonttype'] = 42
plt.rcParams['xtick.labelsize'] = 10
plt.rcParams['ytick.labelsize'] = 10
plt.rcParams['axes.labelsize'] = 12


     |████████████████████████████████| 952kB 2.8MB/s 
  Created wheel for psytrack: filename=psytrack-1.3.0-cp36-none-any.whl size=953537 sha256=f2af1a63100c0822d034cacc37acbb8ca9f786e5e594420fe383985bb90795ab
  Stored in directory: /root/.cache/pip/wheels/bd/92/cd/77769cd11f367beff726d4d5328411b3a3dd3de605400078b6
Successfully built psytrack


In [ ]:
!pip install ibllib
!wget -nc -O "{SPATH}ibl-behavior-data-Dec2019.zip" "https://ndownloader.figshare.com/files/21623715"
!unzip -d "{SPATH}" -n "{SPATH}ibl-behavior-data-Dec2019.zip"


Streaming output truncated to the last 5000 lines.
   creating: ColabFigureData/ibl-behavioral-data-Dec2019/hoferlab/Subjects/SWC_014/2019-08-30/
   creating: ColabFigureData/ibl-behavioral-data-Dec2019/hoferlab/Subjects/SWC_014/2019-08-30/001/
   creating: ColabFigureData/ibl-behavioral-data-Dec2019/hoferlab/Subjects/SWC_014/2019-08-30/001/alf/
  inflating: ColabFigureData/ibl-behavioral-data-Dec2019/hoferlab/Subjects/SWC_014/2019-08-30/001/alf/_ibl_trials.rewardVolume.npy  
  inflating: ColabFigureData/ibl-behavioral-data-Dec2019/hoferlab/Subjects/SWC_014/2019-08-30/001/alf/_ibl_trials.probabilityLeft.npy  
  inflating: ColabFigureData/ibl-behavioral-data-Dec2019/hoferlab/Subjects/SWC_014/2019-08-30/001/alf/_ibl_trials.feedbackType.npy  
  inflating: ColabFigureData/ibl-behavioral-data-Dec2019/hoferlab/Subjects/SWC_014/2019-08-30/001/alf/_ibl_trials.goCue_times.npy  
  inflating: ColabFigureData/ibl-behavioral-data-Dec2019/hoferlab/Subjects/SWC_014/2019-08-30/001/alf/_ibl_trials.stim

In [ ]:
from oneibl.onelight import ONE

ibl_data_path = SPATH + 'ibl-behavioral-data-Dec2019'
current_cwd = os.getcwd()
os.chdir(ibl_data_path)

# Search all sessions that have these dataset types.
required_vars = ['_ibl_trials.choice', '_ibl_trials.contrastLeft',
                 '_ibl_trials.contrastRight','_ibl_trials.feedbackType']
one = ONE()
eids = one.search(required_vars)

mouseData = pd.DataFrame()
for eid in eids:
    lab, _, subject, date, session = eid.split("/")    
    sess_vars = {
        "eid": eid,
        "lab": lab,
        "subject": subject,
        "date": date,
        "session": session,
    }
    mouseData = mouseData.append(sess_vars, sort=True, ignore_index=True)

os.chdir(current_cwd)

/usr/local/lib/python3.6/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [ ]:
#@title processing raw trial data; saved to SPATH directory.
all_vars = ["contrastLeft", "contrastRight", "choice", "feedbackType", "probabilityLeft"]
df = pd.DataFrame()

all_mice = []
for j, s in enumerate(mouseData["subject"].unique()):
    print("\rProcessing " + str(j+1) + " of " + str(len(mouseData["subject"].unique())), end="")
    mouse = mouseData[mouseData["subject"]==s].sort_values(['date', 'session']).reset_index()
    for i, row in mouse.iterrows():
        myVars = {}
        for v in all_vars:
            filename = "_ibl_trials." + v + ".npy"
            var_file = os.path.join(ibl_data_path, row.eid, "alf", filename)
            myVars[v] = list(np.load(var_file).flatten())

        num_trials = len(myVars[v])
        myVars['lab'] = [row.lab]*num_trials
        myVars['subject'] = [row.subject]*num_trials
        myVars['date'] = [row.date]*num_trials
        myVars['session'] = [row.session]*num_trials

        all_mice += [pd.DataFrame(myVars, columns=myVars.keys())]
        
df = pd.concat(all_mice, ignore_index=True)

df = df[df['choice'] != 0]        # dump mistrials
df = df[df['feedbackType'] != 0]  # 3 anomalous trials from ZM_1084, omit
df.loc[np.isnan(df['contrastLeft']), "contrastLeft"] = 0
df.loc[np.isnan(df['contrastRight']), "contrastRight"] = 0
df.loc[df["contrastRight"] < 0, "contrastLeft"] = np.abs(df.loc[df["contrastRight"] < 0, "contrastRight"])
df.loc[df["contrastRight"] < 0, "contrastRight"] = 0  # 81 anomalous trials in CSHL_002, correct
df["answer"] = df["feedbackType"] * df["choice"]      # new column to indicate correct answer
df.loc[df["answer"]==1, "answer"] = 0
df.loc[df["answer"]==-1, "answer"] = 1
df.loc[df["feedbackType"]==-1, "feedbackType"] = 0
df.loc[df["choice"]==1, "choice"] = 0
df.loc[df["choice"]==-1, "choice"] = 1
df.to_csv(SPATH+"ibl_processed.csv", index=False)

Processing 101 of 101

In [ ]:
#@title sanity checks 
print("contrastLeft: ", np.unique(df['contrastLeft']))   # [0, 0.0625, 0.125, 0.25, 0.5, 1.0] and [0.04, 0.08]
print("contrastRight: ", np.unique(df['contrastRight'])) # [0, 0.0625, 0.125, 0.25, 0.5, 1.0] and [0.04, 0.08]
print("choice: ", np.unique(df['choice']))               # [0, 1]
print("feedbackType: ", np.unique(df['feedbackType']))   # [0, 1]
print("answer: ", np.unique(df['answer']))               # [0, 1]

contrastLeft:  [0.     0.04   0.0625 0.08   0.125  0.25   0.5    1.    ]
contrastRight:  [0.     0.04   0.0625 0.08   0.125  0.25   0.5    1.    ]
choice:  [0 1]
feedbackType:  [0 1]
answer:  [0 1]


In [ ]:
#@title Extract Data on Single Mice
ibl_mouse_data_path = SPATH + "ibl_processed.csv"

MOUSE_DF = pd.read_csv(ibl_mouse_data_path)

def getMouse(subject, p=5):
    
    df = MOUSE_DF['subject']==['IBL-T1']   # Restrict data to the subject specified
    
    single_mouse = MOUSE_DF[MOUSE_DF['subject'] == 'IBL-T1']

    cL = np.tanh(p*df['contrastLeft'])/np.tanh(p)   # tanh transformation of left contrasts
    cR = np.tanh(p*df['contrastRight'])/np.tanh(p)  # tanh transformation of right contrasts
    inputs = dict(cL = np.array(cL)[:, None], cR = np.array(cR)[:, None])
    
    dat = dict(
        subject=subject,
        lab=np.unique(df["lab"])[0],
        contrastLeft=np.array(df['contrastLeft']),
        contrastRight=np.array(df['contrastRight']),
        date=np.array(df['date']),
        dayLength=np.array(df.groupby(['date','session']).size()),
        correct=np.array(df['feedbackType']),
        answer=np.array(df['answer']),
        probL=np.array(df['probabilityLeft']),
        inputs = inputs,
        y = np.array(df['choice'])
    )
    
    return dat 